In [4]:
pip install Unidecode

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
from unidecode import unidecode

In [8]:
import pandas as pd
import requests
import unicodedata
from unidecode import unidecode
import re
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [9]:
url_neighborhood = 'https://www.estadosecapitaisdobrasil.com/listas/lista-dos-bairros-de-sao-paulo/'

In [10]:
response = requests.get(url_neighborhood)
html = response.content
soup = BeautifulSoup(html)
neighborhood_sp = []
for neighb in soup.find_all('li')[24:]:
    neighborhood_sp.append(unicodedata.normalize('NFKD', neighb.text).encode('ASCII', 'ignore').decode('utf-8').strip().lower().replace(' ','+'))

In [11]:
print(neighborhood_sp)

['atividades+permitidas+no+mei', 'agua+rasa', 'alto+de+pinheiros', 'anhanguera', 'aricanduva', 'artur+alvim', 'barra+funda', 'bela+vista', 'belem', 'bom+retiro', 'brasilandia', 'butanta', 'cachoeirinha', 'cambuci', 'campo+belo', 'campo+grande', 'campo+limpo', 'cangaiba', 'capao+redondo', 'carrao', 'casa+verde', 'cidade+ademar', 'cidade+dutra', 'cidade+lider', 'cidade+lider', 'cidade+tiradentes', 'consolacao', 'cursino', 'ermelino+matarazzo', 'freguesia+do+o', 'grajau', 'guaianases', 'iguatemi', 'ipiranga', 'itaim+bibi', 'itaim+paulista', 'itaquera', 'jabaquara', 'jacana', 'jaguara', 'jaguare', 'jaragua', 'jardim+angela', 'jardim+helena', 'jardim+paulista', 'jardim+sao+luis', 'lapa', 'liberdade', 'limao', 'mandaqui', 'marsilac', 'moema', 'mooca', 'morumbi', 'parelheiros', 'pari', 'parque+do+carmo', 'penha', 'perdizes', 'pinheiros', 'ponte+rasa', 'raposo+tavares', 'republica', 'rio+pequeno', 'sacoma', 'santa+cecilia', 'santana', 'santo+amaro', 'sao+domingos', 'sao+lucas', 'sao+mateus', '

In [12]:
#this code get the streets from neighborhood selected previously
df_street = pd.DataFrame()
for neighborhood in neighborhood_sp:
    for pag in range(1,20):
        try:
            url = fr'https://cep.guiamais.com.br/busca/{neighborhood}-sao+paulo-sp?page={pag}'
            response = requests.get(url)
            html = response.content
            soup = BeautifulSoup(html)
            table = soup.find_all('table')
            data = pd.read_html(str(table))[0]
            df_street = df_street.append(data, ignore_index=True)
        except:
            pass

In [13]:
df_street.head()

,LOGRADOURO,BAIRRO,CIDADE/ESTADO,BAIRRO.1,CEP
0,Rua 03 de Outubro (Ch Três Meninas),Jardim Helena,"São Paulo, SP","Jardim Helena, São Paulo, SP",08090-284
1,Rua 13 de Maio,Cantinho do Céu,"São Paulo, SP","Cantinho do Céu, São Paulo, SP",04849-529
2,Viela 16,Parque São José,"São Paulo, SP","Parque São José, São Paulo, SP",04843-425
3,Rua 18 de Abril,Cidade Antônio Estevão de Carvalho,"São Paulo, SP","Cidade Antônio Estevão de Carvalho, São Paulo, SP",08226-021
4,Travessa 19 de Agosto,Jardim Maria Estela,"São Paulo, SP","Jardim Maria Estela, São Paulo, SP",04180-112


In [15]:
#saving the dataframe 
df_street.to_csv('C:/Users/smgma/OneDrive/Sonia/Ironhack/Aulas/Modulo2/Projetos/Projeto3/df_street.csv', index = False)

In [17]:
def transform_research_neighborhood(row):
    row = unidecode(row).lower()
    row = row.replace(' ','-')
    return row

In [18]:
def transform_street(row):
    row = unidecode(row).lower()
    row = row.replace('- ate ','')
    row = row.replace(' - lado impar','')
    row = row.replace(' - lado par','')
    row = row.replace(' ','-')
    return row

In [19]:
def clean_street_variable(row):
    row = row.replace('--de-','')
    row = row.split('-a-')[0]
    row = row.split('-ao-')[0]   
    row = row.replace('/','-')
    return row

In [20]:
df_street['BAIRRO'] = df_street['BAIRRO'].apply(transform_research_neighborhood)
df_street['LOGRADOURO'] = df_street['LOGRADOURO'].apply(transform_street)

In [21]:
df_street.head()

,LOGRADOURO,BAIRRO,CIDADE/ESTADO,BAIRRO.1,CEP
0,rua-03-de-outubro-(ch-tres-meninas),jardim-helena,"São Paulo, SP","Jardim Helena, São Paulo, SP",08090-284
1,rua-13-de-maio,cantinho-do-ceu,"São Paulo, SP","Cantinho do Céu, São Paulo, SP",04849-529
2,viela-16,parque-sao-jose,"São Paulo, SP","Parque São José, São Paulo, SP",04843-425
3,rua-18-de-abril,cidade-antonio-estevao-de-carvalho,"São Paulo, SP","Cidade Antônio Estevão de Carvalho, São Paulo, SP",08226-021
4,travessa-19-de-agosto,jardim-maria-estela,"São Paulo, SP","Jardim Maria Estela, São Paulo, SP",04180-112


In [22]:
qa_streets_search = df_street['LOGRADOURO'] + '-' + df_street['BAIRRO']

In [23]:
qa_streets_search = qa_streets_search.apply(clean_street_variable)

In [24]:
#return a list with streets to search in quinto andar website
list_streets = []
for rua in range(qa_streets_search.size):
    list_streets.append(qa_streets_search[rua])

In [25]:
list_streets

['rua-03-de-outubro-(ch-tres-meninas)-jardim-helena',
 'rua-13-de-maio-cantinho-do-ceu',
 'viela-16-parque-sao-jose',
 'rua-18-de-abril-cidade-antonio-estevao-de-carvalho',
 'travessa-19-de-agosto-jardim-maria-estela',
 'rua-2-de-fevereiro-cidade-nova-heliopolis',
 'rua-21-de-abril-bras',
 'rua-23-de-novembro-vila-progresso-(zona-leste)',
 'avenida-24-de-agosto-conjunto-habitacional-fazenda-do-carmo',
 'travessa-26-de-outubro-jardim-nazareth',
 'rua-29-lotm-cananeia',
 'rua-29-de-julho-(vila-helena)-jardim-sao-martinho',
 'rua-3-jardim-jau-(zona-sul)',
 'viaduto-31-de-marco-liberdade',
 'rua-5-de-outubro-chacara-gaivotas',
 'rua-7-g-cidade-tiradentes',
 'travessa-713-guaianazes',
 'rua-9-de-setembro-chacara-gaivotas',
 'travessa',
 'caminho',
 'estrada',
 'praca',
 'passagem',
 'passagem',
 'rua',
 'rua-03-de-outubro-(ch-tres-meninas)-jardim-helena',
 'rua-13-de-maio-cantinho-do-ceu',
 'viela-16-parque-sao-jose',
 'rua-18-de-abril-cidade-antonio-estevao-de-carvalho',
 'travessa-19-de-a

In [81]:
len(list_streets)

14320

In [83]:
len(list(set(list_streets)))

12824

In [85]:
list_streets=list(set(list_streets)) 

In [88]:
list_streets

['rua-maria-celestina-fraga-parque-grajau',
 'rua-correia-salgado-ipiranga',
 'rua-guaruma-parque-jabaquara',
 'rua-caramuru-247-248',
 'alameda-caviunas-recanto-campo-belo',
 'rua-engenheiro-toledo-malta-saude',
 'travessa-laranja-cravo-ermelino-matarazzo',
 'rua-jose-santana-vila-reis',
 'avenida-morumbi-5055-5056',
 'rua-jose-inague-vila-medeiros',
 'rua-simpliciano-ferreira-jaragua',
 'rua-baltazar-lisboa-170-vila-mariana',
 'rua-botupuca-cambuci',
 'rua-isidro-tinoco-147-tatuape',
 'rua-felipe-gadelha-santana',
 'rua-iacri-vila-carrao',
 'rua-canto-bonito-vila-andrade',
 'avenida-gustavo-berthier-rio-pequeno',
 'travessa-lucas-alber-vila-guilherme',
 'avenida-doutor-gastao-vidigal-2001-2002-vila-leopoldina',
 'rua-alvarenga-1101-1102',
 'rua-alexandre-calame-jaguare',
 'rua-da-consolacao-222-consolacao',
 'avenida-brigadeiro-luis-antonio-1089-bela-vista',
 'rua-martinopolis-cidade-dutra',
 'rua-paulo-arentino-conjunto-city-jaragua',
 'rua-scuvero-cambuci',
 'rua-mamud-rahd-trememb

In [ ]:

list_apartment_photo = []

for streets in list_streets:
    url = f'https://www.quintoandar.com.br/comprar/imovel/{streets}-sao-paulo-sp-brasil/'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    
    #advertising [2], [10] ignore them
    for tag_card in [x for x in range(18) if x != 2 and x != 10]:
        try:
            cards = soup.find_all('div', attrs={'class':'sc-1qwl1yl-0 igVsBW'})[tag_card]
            
            photo = cards.find('div' ,attrs={'class':'sc-1nwwxw0-0 drvPOq'}).img['src']


            apartment = {  
                            'photo': photo
                         }

            list_apartment_photo.append(apartment)
        except:
            pass

ConnectionError: HTTPSConnectionPool(host='www.quintoandar.com.br', port=443): Max retries exceeded with url: /comprar/imovel/travessa-calado-liberdade-sao-paulo-sp-brasil/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002957F401340>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [26]:
df_qa_id = pd.DataFrame(list_apartment_photo)

NameError: name 'list_apartment_photo' is not defined

In [11]:
df_qa_photo = pd.read_csv('C:/Users/sonia/OneDrive/Sonia/Ironhack/Aulas/Modulo2/Projetos/Projeto3/df_qa_photo.csv')

In [12]:
df_qa_photo

,photo
0,/img/med/893161472-288.3220545571478WhatsAppIm...
1,/img/med/893169262-830.8323378785544WhatsAppIm...
2,/img/med/893161268-855.618460638472603102020IM...
3,/img/med/893104875-761.9647653640459IMG2020071...
4,/img/med/893162831-245.25879451646872020093011...
...,...
146769,/img/med/893166994-392.9399107456047WhatsAppIm...
146770,/img/med/892991514-442.0603251928376IMG1868.jpg
146771,/img/med/893152230-343.35913766449556WhatsAppI...
146772,/img/med/893169031-702.3878825821203IMG2020100...


In [5]:
def pick_id(row):
    row = row.split('/img/med/')[1]
    return row.split('-')[0]

In [14]:
df_qa_photo['ID'] = df_qa_photo['photo'].apply(pick_id)
df_qa_photo

,photo,ID
0,/img/med/893161472-288.3220545571478WhatsAppIm...,893161472
1,/img/med/893169262-830.8323378785544WhatsAppIm...,893169262
2,/img/med/893161268-855.618460638472603102020IM...,893161268
3,/img/med/893104875-761.9647653640459IMG2020071...,893104875
4,/img/med/893162831-245.25879451646872020093011...,893162831
...,...,...
146769,/img/med/893166994-392.9399107456047WhatsAppIm...,893166994
146770,/img/med/892991514-442.0603251928376IMG1868.jpg,892991514
146771,/img/med/893152230-343.35913766449556WhatsAppI...,893152230
146772,/img/med/893169031-702.3878825821203IMG2020100...,893169031


In [15]:
df_qa_photo = df_qa_photo.drop_duplicates()

In [16]:
df_qa_photo

,photo,ID
0,/img/med/893161472-288.3220545571478WhatsAppIm...,893161472
1,/img/med/893169262-830.8323378785544WhatsAppIm...,893169262
2,/img/med/893161268-855.618460638472603102020IM...,893161268
3,/img/med/893104875-761.9647653640459IMG2020071...,893104875
4,/img/med/893162831-245.25879451646872020093011...,893162831
...,...,...
145618,/img/med/893094608-985.3031619927204QuintoAnda...,893094608
145651,/img/med/893140171-571.6132158103559QuintoAnda...,893140171
145661,/img/med/892829417-404.18812458393296MG4004.JPG,892829417
145662,/img/med/892896687-32.5807846352848FOTO7510.jpg,892896687


In [7]:

df_qa_photo['ID'] = df_qa_photo['photo'].apply(pick_id)
df_qa_photo = df_qa_photo['ID']
df_qa_photo = df_qa_photo.drop_duplicates()

In [8]:
df_qa_photo.to_csv('C:/Users/sonia/OneDrive/Sonia/Ironhack/Aulas/Modulo2/Projetos/Projeto3/df_qa_id2.csv', index = False)

In [10]:
df_qa_photo

0         893161472
1         893169262
2         893161268
3         893104875
4         893162831
            ...    
145618    893094608
145651    893140171
145661    892829417
145662    892896687
145673    893032638
Name: ID, Length: 3362, dtype: object

In [17]:
list_apartment = []

for cod in df_qa_photo['ID']:
    url = f'https://www.quintoandar.com.br/imovel/{cod}'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    

    
    try:
        endereco = re.findall(r'address:(.*)availabilityType',soup.text)
        endereco = endereco[0].replace('},"e"],','')
        endereco = endereco.replace('[{','')
        endereco = endereco.split(',')            
        
        
        
        
        tipo_imovel = soup.find('h1').text
        logradouro = soup.find('p',attrs={'class':'sc-bdVaJa gZyMJB'}).text.split(',')[0]     
        bairro = soup.find('p',attrs={'class':'sc-bdVaJa gZyMJB'}).text.split(',')[1]                
        cidade = soup.find('p',attrs={'class':'sc-bdVaJa gZyMJB'}).text.split(',')[2]         
        metragem = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[0].text
        quartos = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[1].text
        banheiros = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[2].text
        vaga = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[3].text
        andar = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[4].text
        mobilia = soup.find_all('span',attrs={'class':'sc-bdVaJa eNFcTE'})[6].text
        aluguel = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 iRCHiJ'})[0].text
        condominio = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 bkKyCN'})[0].text
        iptu = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 bkKyCN'})[1].text
        preco_total = soup.find_all('span',attrs={'class':'ek9a7p-0 djWxJW sc-11qijje-0 iRCHiJ'})[1].text
    
    
    
        apartment = { 
                        'codigo' : cod,
                        'endereco' : endereco,
                        'tipo_imovel' : tipo_imovel,
                        'logradouro' : logradouro,
                        'bairro' : bairro,
                        'cidade' : cidade,
                        'metragem' : metragem,
                        'quartos': quartos,
                        'banheiros': banheiros,
                        'vaga': vaga,
                        'andar' : andar,
                        'mobilia' : mobilia,
                        'aluguel': aluguel,
                        'condominio' : condominio,
                        'iptu' : iptu,
                        'preco_total' : preco_total,
                     }

        list_apartment.append(apartment)   
            
            
    except:
        pass

In [18]:

df_qa_apartment_raw = pd.DataFrame(list_apartment)

In [19]:
df_qa_apartment_raw.to_csv('C:/Users/sonia/OneDrive/Sonia/Ironhack/Aulas/Modulo2/Projetos/Projeto3/df_qa_apartment_raw.csv', index = False)

In [67]:
df_qa_apartment_raw = pd.read_csv('C:/Users/sonia/OneDrive/Sonia/Ironhack/Aulas/Modulo2/Projetos/Projeto3/df_qa_apartment_raw.csv')
df_qa_apartment_raw

,codigo,endereco,tipo_imovel,logradouro,bairro,cidade,metragem,quartos,banheiros,vaga,andar,mobilia,aluguel,condominio,iptu,preco_total
0,893156675,"['{street:""Rua Damiana""', 'number:""62""', 'neig...","Casa para alugar com 4 quartos, 222m²",Rua Damiana,Vila Jacuí,São Paulo,222 m²,4 quartos,3 banheiros,7 vagas,-,Sem mobília,R$ 5.320,Incluso,R$ 30,R$ 5.438
1,893161607,"['{street:""Rua Aiuba""', 'number:""03""', 'neighb...","Apartamento para alugar com 3 quartos, 51m²",Rua Aiuba,Vila Jacuí,São Paulo,51 m²,3 quartos,1 banheiro,1 vaga,3º andar,Sem mobília,R$ 1.300,R$ 363,R$ 0,R$ 1.680
2,893163046,"['{street:""Rua Coimbra""', 'number:""583""', 'nei...","Apartamento para alugar com 2 quartos, 57m²",Rua Coimbra,Centro,Diadema,57 m²,2 quartos,1 banheiro,1 vaga,1º andar,Sem mobília,R$ 1.200,R$ 560,R$ 85,R$ 1.861
3,893146117,"['{street:""Rua Guaruva""', 'number:""537""', 'nei...","Casa para alugar com 4 quartos, 200m²",Rua Guaruva,Cidade Dutra,São Paulo,200 m²,4 quartos,3 banheiros,3 vagas,-,Sem mobília,R$ 2.550,R$ 0,R$ 125,R$ 2.769
4,893046057,"['{street:""Rua Guilherme de Almeida""', 'number...","Casa para alugar com 1 quarto, 45m²",Rua Guilherme de Almeida,Conceição,Diadema,45 m²,1 quarto,1 banheiro,1 vaga,-,Sem mobília,R$ 1.149,R$ 0,R$ 0,R$ 1.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,893146044,"['{street:""Rua Ibitirama""', 'number:""1820""', '...","Apartamento para alugar com 2 quartos, 58m²",Rua Ibitirama,Vila Prudente,São Paulo,58 m²,2 quartos,1 banheiro,Sem vaga,3º andar,Mobiliado,R$ 1.850,R$ 600,R$ 0,R$ 2.511
1409,893125919,"['{street:""Rua Domingos Afonso""', 'number:""460...","Apartamento para alugar com 3 quartos, 126m²",Rua Domingos Afonso,Vila Santa Clara,São Paulo,126 m²,3 quartos,2 banheiros,2 vagas,21º andar,Sem mobília,R$ 2.910,R$ 780,R$ 222,R$ 4.007
1410,893094608,"['{street:""Rua Giovani Pattoli""', 'number:""606...","Apartamento para alugar com 2 quartos, 83m²",Rua Giovani Pattoli,Jardim Avelino,São Paulo,83 m²,2 quartos,2 banheiros,2 vagas,12º andar,Sem mobília,R$ 2.600,R$ 715,R$ 259,R$ 3.659
1411,892829417,"['{street:""R. Ibitirama""', 'number:""1556""', 'n...","Apartamento para alugar com 2 quartos, 75m²",R. Ibitirama,Vila Prudente,São Paulo,75 m²,2 quartos,2 banheiros,1 vaga,1º andar,Sem mobília,R$ 1.690,R$ 680,R$ 30,R$ 2.456


In [68]:
def split_elements(row):
    row = re.sub('\'|\[|\]|"|{','',row).split(',')
    return row

In [69]:
df_qa_apartment_raw['endereco'] = df_qa_apartment_raw['endereco'].apply(split_elements)

In [70]:
addresses = df_qa_apartment_raw.endereco.apply(lambda x : pd.Series(x))
addresses

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,street:Rua Damiana,number:62,neighborhood:Jardim Matarazzo,city:São Paulo,zipCode:03813-260,stateName:São Paulo,stateAcronym:SP,lat:-23.4886626,lng:-46.4671373,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,street:Rua Aiuba,number:03,neighborhood:Parque Cruzeiro do Sul,city:São Paulo,zipCode:08070-100,stateName:São Paulo,stateAcronym:SP,lat:-23.4919344,lng:-46.4644629,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,street:Rua Coimbra,number:583,neighborhood:Centro,city:Diadema,zipCode:09910-120,stateName:São Paulo,stateAcronym:SP,lat:-23.6970916,lng:-46.6302367,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,street:Rua Guaruva,number:537,neighborhood:Cidade Dutra,city:São Paulo,zipCode:04807-030,stateName:São Paulo,stateAcronym:SP,lat:-23.7124263,lng:-46.7000525,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,street:Rua Guilherme de Almeida,number:11,neighborhood:Conceição,city:Diadema,zipCode:09993-000,stateName:São Paulo,stateAcronym:SP,lat:-23.7014963,lng:-46.6244207,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,street:Rua Ibitirama,number:1820,neighborhood:Vila Prudente,city:São Paulo,zipCode:03133-000,stateName:São Paulo,stateAcronym:SP,lat:-23.5958181,lng:-46.578412,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,street:Rua Domingos Afonso,number:460,neighborhood:Vila Santa Clara,city:São Paulo,zipCode:03161-090,stateName:São Paulo,stateAcronym:SP,lat:-23.5780948,lng:-46.5627982,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,street:Rua Giovani Pattoli,number:606,neighborhood:Jardim Avelino,city:São Paulo,zipCode:03227-090,stateName:São Paulo,stateAcronym:SP,lat:-23.584413,lng:-46.5706558,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1411,street:R. Ibitirama,number:1556,neighborhood:Vila Prudente,city:São Paulo,zipCode:03134-002,stateName:São Paulo,stateAcronym:SP,lat:-23.5938459,lng:-46.5792483,country:Brazil},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
def find_numero(row):
    numero = re.findall('\d+',row)
    return numero

In [72]:

def find_cep(row):
    return re.findall('\d+-\d+', row)

In [73]:
def find_lat_long(row):
    if type(row) == str:
        return re.findall('\d+.\d+',row)
    else:
        pass

In [74]:
def acessa_lista(row):
    if type(row) == list:
        for x in row:
            return x
    else:
        return row

In [75]:
def clean_metragem_quartos(row):
    numero = re.findall('\d+', row)
    return numero

In [76]:

def clean_precos(row):
    if type(row) == str:
        row = row.replace('.','')
        row = row.replace(',','')
        valor = re.findall('\d+', row)
        return valor
    else:
        return row

In [77]:
def acessa_lista_int(row):
    if type(row) == list:
        for x in row:
            return int(x)
    else:
        return row

In [78]:
def negative_numbers(row):
    if type(row) == str:
        row = float(row)*-1
        return row
    else:
        return row

In [79]:
lista_tipologias = ['apartamento', 'studio e kitnet','casa']

def tipologia(row, lista_tipologias=lista_tipologias):
    for tipo in lista_tipologias:
        if re.findall(tipo, row, flags=re.IGNORECASE):
            return tipo
        
    return 'outro'

In [80]:
addresses['longitude'] = addresses.iloc[:,[8]]
addresses['latitude'] = addresses.iloc[:,[7]]

In [81]:
addresses['numero'] = addresses.iloc[:,[1]]
addresses['cep'] = addresses.iloc[:,[4]]
addresses = addresses[['numero','cep','longitude','latitude']]


In [82]:
addresses['numero'] = addresses['numero'].apply(find_numero)
addresses['cep'] = addresses['cep'].apply(find_cep)
addresses['longitude'] = addresses['longitude'].apply(find_lat_long)
addresses['latitude'] = addresses['latitude'].apply(find_lat_long)

C:\Users\sonia\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\sonia\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sonia\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [83]:
addresses['numero'] = addresses['numero'].apply(acessa_lista)
addresses['cep'] = addresses['cep'].apply(acessa_lista)
addresses['longitude'] = addresses['longitude'].apply(acessa_lista)
addresses['latitude'] = addresses['latitude'].apply(acessa_lista)

C:\Users\sonia\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\sonia\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sonia\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [84]:
df_qa_apartment = pd.concat([df_qa_apartment_raw, addresses], axis=1)
df_qa_apartment = df_qa_apartment.drop(columns='endereco')

In [85]:

df_qa_apartment.head()

,codigo,tipo_imovel,logradouro,bairro,cidade,metragem,quartos,banheiros,vaga,andar,mobilia,aluguel,condominio,iptu,preco_total,numero,cep,longitude,latitude
0,893156675,"Casa para alugar com 4 quartos, 222m²",Rua Damiana,Vila Jacuí,São Paulo,222 m²,4 quartos,3 banheiros,7 vagas,-,Sem mobília,R$ 5.320,Incluso,R$ 30,R$ 5.438,62,03813-260,46.4671373,23.4886626
1,893161607,"Apartamento para alugar com 3 quartos, 51m²",Rua Aiuba,Vila Jacuí,São Paulo,51 m²,3 quartos,1 banheiro,1 vaga,3º andar,Sem mobília,R$ 1.300,R$ 363,R$ 0,R$ 1.680,03,08070-100,46.4644629,23.4919344
2,893163046,"Apartamento para alugar com 2 quartos, 57m²",Rua Coimbra,Centro,Diadema,57 m²,2 quartos,1 banheiro,1 vaga,1º andar,Sem mobília,R$ 1.200,R$ 560,R$ 85,R$ 1.861,583,09910-120,46.6302367,23.6970916
3,893146117,"Casa para alugar com 4 quartos, 200m²",Rua Guaruva,Cidade Dutra,São Paulo,200 m²,4 quartos,3 banheiros,3 vagas,-,Sem mobília,R$ 2.550,R$ 0,R$ 125,R$ 2.769,537,04807-030,46.7000525,23.7124263
4,893046057,"Casa para alugar com 1 quarto, 45m²",Rua Guilherme de Almeida,Conceição,Diadema,45 m²,1 quarto,1 banheiro,1 vaga,-,Sem mobília,R$ 1.149,R$ 0,R$ 0,R$ 1.167,11,09993-000,46.6244207,23.7014963


In [86]:
df_qa_apartment['tipo_imovel'] = df_qa_apartment['tipo_imovel'].apply(tipologia)

In [87]:
df_qa_apartment['metragem'] = df_qa_apartment['metragem'].apply(clean_metragem_quartos)
df_qa_apartment['quartos'] = df_qa_apartment['quartos'].apply(clean_metragem_quartos)
df_qa_apartment['banheiros'] = df_qa_apartment['banheiros'].apply(clean_metragem_quartos)

In [88]:
df_qa_apartment['vaga'] = df_qa_apartment['vaga'].apply(clean_metragem_quartos)
df_qa_apartment['andar'] = df_qa_apartment['andar'].apply(clean_metragem_quartos)
df_qa_apartment['aluguel'] = df_qa_apartment['aluguel'].apply(clean_precos)
df_qa_apartment['condominio'] = df_qa_apartment['condominio'].apply(clean_precos)
df_qa_apartment['iptu'] = df_qa_apartment['iptu'].apply(clean_precos)
df_qa_apartment['preco_total'] = df_qa_apartment['preco_total'].apply(clean_precos)

In [89]:
df_qa_apartment['longitude'] = df_qa_apartment['longitude'].apply(negative_numbers)
df_qa_apartment['latitude'] = df_qa_apartment['latitude'].apply(negative_numbers)

ValueError: could not convert string to float: '08210-500'

In [90]:
df_qa_apartment['aluguel'] = df_qa_apartment['aluguel'].apply(acessa_lista)
df_qa_apartment['condominio'] = df_qa_apartment['condominio'].apply(acessa_lista)
df_qa_apartment['iptu'] = df_qa_apartment['iptu'].apply(acessa_lista)
df_qa_apartment['preco_total'] = df_qa_apartment['preco_total'].apply(acessa_lista)
df_qa_apartment['metragem'] = df_qa_apartment['metragem'].apply(acessa_lista)
df_qa_apartment['quartos'] = df_qa_apartment['quartos'].apply(acessa_lista)
df_qa_apartment['banheiros'] = df_qa_apartment['banheiros'].apply(acessa_lista)
df_qa_apartment['vaga'] = df_qa_apartment['vaga'].apply(acessa_lista)
df_qa_apartment['andar'] = df_qa_apartment['andar'].apply(acessa_lista)

In [91]:

df_qa_apartment.head(10)

,codigo,tipo_imovel,logradouro,bairro,cidade,metragem,quartos,banheiros,vaga,andar,mobilia,aluguel,condominio,iptu,preco_total,numero,cep,longitude,latitude
0,893156675,casa,Rua Damiana,Vila Jacuí,São Paulo,222,4,3,7,None,Sem mobília,5320,None,30,5438,62,03813-260,46.4671373,23.4886626
1,893161607,apartamento,Rua Aiuba,Vila Jacuí,São Paulo,51,3,1,1,3,Sem mobília,1300,363,0,1680,03,08070-100,46.4644629,23.4919344
2,893163046,apartamento,Rua Coimbra,Centro,Diadema,57,2,1,1,1,Sem mobília,1200,560,85,1861,583,09910-120,46.6302367,23.6970916
3,893146117,casa,Rua Guaruva,Cidade Dutra,São Paulo,200,4,3,3,None,Sem mobília,2550,0,125,2769,537,04807-030,46.7000525,23.7124263
4,893046057,casa,Rua Guilherme de Almeida,Conceição,Diadema,45,1,1,1,None,Sem mobília,1149,0,0,1167,11,09993-000,46.6244207,23.7014963
5,893164553,casa,Rua José André Pattini,Cidade Dutra,São Paulo,95,2,1,2,None,Sem mobília,1870,None,117,2055,368,04809-260,46.693379,23.7138078
6,893043524,casa,Rua Leonel Marques Bernardes,Interlagos,São Paulo,434,4,4,9,None,Mobiliado,8000,None,998,9291,75,04789-050,46.7070911,23.7068704
7,893155448,casa,Rua Itaueira,Itaquera,São Paulo,50,1,1,None,None,Sem mobília,650,0,47,708,25,08240-174,46.4528411,23.5239518
8,893168337,casa,Travessa do Anfiguri,Vila Jacuí,São Paulo,45,1,2,None,None,Sem mobília,880,0,0,895,171,08050-495,46.4612294,23.5126971
9,893164206,casa,Rua Dia Branco,Itaquera,São Paulo,25,1,1,None,None,Sem mobília,750,0,0,763,18,08223-400,46.4710423,23.5245083


In [92]:
df_qa_apartment.to_csv('C:/Users/sonia/OneDrive/Sonia/Ironhack/Aulas/Modulo2/Projetos/Projeto3/df_qa_apartament.csv', index = False)

In [59]:
dict_utm = {}

for row in range(df_quinto_andar.shape[0]):

    lat = df_quinto_andar['latitude'][row]
    long = df_quinto_andar['longitude'][row]
    try:
        y, x, z, k = utm.from_latlon(lat, long)
        dict_row_utm = {}
        dict_row_utm = {'latitude_utm': round(x,2) , 'longitude_utm' : round(y,2)}
        dict_utm[row] = dict_row_utm
    except:
        dict_utm[row] = dict_row_utm
        dict_row_utm = {'latitude_utm': 0 , 'longitude_utm' : 0}

NameError: name 'df_quinto_andar' is not defined

In [ ]:

df_quinto_andar_utm = pd.DataFrame.from_dict(dict_utm, orient='index')
df_qa_clean =pd.concat([df_quinto_andar, df_quinto_andar_utm], axis=1)